In [10]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv


# API 키 정보 로드
load_dotenv()


True

In [5]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("SeniorMTS-RAG")


LangSmith 추적을 시작합니다.
[프로젝트명]
SeniorMTS-RAG


In [68]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser
import os
from dotenv import load_dotenv
from langchain_teddynote.korean import stopwords
from langchain_upstage import UpstageEmbeddings
from langchain_teddynote.community.pinecone import init_pinecone_index, PineconeKiwiHybridRetriever
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever




########## 1. 도구를 정의합니다 ##########


### 1-2. PDF 문서 검색 도구 (Retriever) ###

pinecone_cycle = init_pinecone_index(

    index_name="seniormts",  # Pinecone 인덱스 이름
    namespace="cyclereports",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./cyclereports_sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"
    ),  # Dense Embedder
    top_k=3,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

# 검색기 생성
cycle_retriever = PineconeKiwiHybridRetriever(**pinecone_cycle)

pinecone_stock = init_pinecone_index(

    index_name="seniormts",  # Pinecone 인덱스 이름
    namespace="stockreports",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./stockreports_sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"
    ),  # Dense Embedder
    top_k=4,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

# 검색기 생성
stock_retriever = PineconeKiwiHybridRetriever(**pinecone_stock)

pinecone_news = init_pinecone_index(

    index_name="seniormts",  # Pinecone 인덱스 이름
    namespace="stocknews",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./stocknews_sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"
    ),  # Dense Embedder
    top_k=3,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

# 검색기 생성
news_retriever = PineconeKiwiHybridRetriever(**pinecone_news)


### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

cycle_search_tool = create_retriever_tool(
    cycle_retriever,
    name="cycle_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information about the economic cycle",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

stockinformation_search_tool = create_retriever_tool(
    stock_retriever,
    name="stock_information_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information about the stock in question",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

newsinformation_search_tool = create_retriever_tool(
    news_retriever,
    name="stock_information_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information about the news related to the stock I asked about",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
tools = [search, cycle_search_tool,stockinformation_search_tool,newsinformation_search_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the '_search tool' for searching  relevent information. "
            "If you can't find the information from the  relevent, use the `search` tool for searching information from the web."
            "You must always respond in the language you are given."
            "You must provide accurate information at all times. If uncertain, do not fabricate responses; instead, firmly state that you do not know."
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "삼성전자 살까 말까"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)


# 최종 답변 추출 및 출력
final_answer = None
for step in response:
    # 각 단계의 출력을 처리
    if isinstance(step, dict) and 'output' in step:
        final_answer = step['output']

# 최종 답변 출력
if final_answer:
    print(final_answer)
else:
    print('답변을 생성하지 못했습니다.')

[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'cyclereports': {'vector_count': 842},
                'stocknews': {'vector_count': 21},
                'stockreports': {'vector_count': 146}},
 'total_vector_count': 1009}
[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'cyclereports': {'vector_count': 842},
                'stocknews': {'vector_count': 21},
                'stockreports': {'vector_count': 146}},
 'total_vector_count': 1009}
[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'cyclereports': {'vector_count': 842},
                'stocknews': {'vector_count': 21},
                'stockreports': {'vector_count': 146}},
 'total_vector_count': 1009}


In [71]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "삼성전자 살까 말까"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)


# 최종 답변 추출 및 출력
final_answer = None
for step in response:
    # 각 단계의 출력을 처리
    if isinstance(step, dict) and 'output' in step:
        final_answer = step['output']

# 최종 답변 출력
if final_answer:
    print(final_answer)
else:
    print('답변을 생성하지 못했습니다.')

삼성전자 주식에 대한 최근 정보와 뉴스는 다음과 같습니다:

1. **주식 정보**: 삼성전자의 최근 주가는 비교적 저평가된 상태로 보입니다. 이는 엔비디아와의 협력 후 기대감이 커지고 있는 상황에서 주가가 하락한 것에 기인합니다. 삼성전자는 3분기 실적 발표에서 기대치를 밑돌았으며, 특히 반도체 사업 부문에서의 수익이 감소했습니다. 그러나 HBM(고대역폭 메모리) 시장 진입과 관련된 긍정적인 전망도 있습니다.

2. **관련 뉴스**: 삼성전자는 최근 3분기 실적 발표에서 기대에 미치지 못하는 성과를 보였으며, 특히 반도체 부문에서의 수익 감소가 두드러졌습니다. 그러나 HBM 시장에서의 경쟁력 강화와 엔비디아와의 협력으로 인해 향후 실적 개선에 대한 기대감이 존재합니다. 또한, 삼성전자는 HBM3E 제품의 공급을 확대할 계획이며, 내년에는 HBM4 로드맵도 발표할 예정입니다.

삼성전자 주식의 매수 여부는 개인의 투자 성향과 시장 전망에 따라 다를 수 있습니다. 현재 주가가 저평가된 상태로 보이지만, 반도체 시장의 불확실성과 경쟁 상황을 고려해야 합니다. 투자 결정을 내리기 전에 추가적인 시장 분석과 전문가의 의견을 참고하는 것이 좋습니다.


In [66]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='삼성전자 이재용 회장이 중국으로 이민 갔다는데 맞아?', additional_kwargs={}, response_metadata={}), AIMessage(content='이재용 삼성전자 회장이 중국으로 이민 갔다는 정보는 확인되지 않았습니다. 최근 이재용 회장은 서울에서 리창 중국 총리와 만남을 가졌으며, 이는 한·일·중 정상회담을 위해 방한한 리 총리와의 만남이었습니다. 이민과 관련된 소식은 없습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='확실해? 내가 아는 소식통에 따르면 정말 정확한 정보인데', additional_kwargs={}, response_metadata={}), AIMessage(content='이재용 삼성전자 회장이 중국으로 이민 갔다는 정보는 확인되지 않았습니다. 최근 뉴스에 따르면, 이재용 회장은 서울에서 리창 중국 총리와 만남을 가졌으며, 이는 한·일·중 정상회담을 위해 방한한 리 총리와의 만남이었습니다. 이민과 관련된 소식은 전혀 없습니다. 따라서 이민에 대한 정보는 잘못된 것으로 보입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='오케이 믿을게, 넌 대단한 아이니까', additional_kwargs={}, response_metadata={}), AIMessage(content='고맙습니다! 언제든지 궁금한 점이 있으면 물어보세요. 항상 도움을 드리겠습니다.', additional_kwargs={}, response_metadata={})])}